# Reconocimiento de señales de tráfico
### Máster Universitario en Ingeniería computacional y matemática - Área de inteligencia artificial
### Antonio González Hidalgo (agonzalezhidalgo@uoc.edu)

Librería estándar con funciones y métodos usandos a lo largo del TFM

In [2]:
# https://docs.python.org/3/library/os.html
import os

# http://scikit-image.org/docs/stable/api/api.html
import skimage

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://docs.python.org/3/library/csv.html
import csv

# https://matplotlib.org/api/index.html
import matplotlib

# https://matplotlib.org/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot
import matplotlib.pyplot as plt

## Métodos privados

In [3]:
# Devuelve una colección de directorios de la ruta
def __get_directories(data_dir):
    
    directories = []
    
    if os.path.exists(data_dir):
        
        #Buscamos todos los directorios de la ruta
        for d in os.listdir(data_dir):
            if os.path.isdir(os.path.join(data_dir, d)):
                directories.append(d)
                
    else:
        print("path doesn't exists")
    return directories

In [4]:
# Obtenemos el índice inicial y final de una label específica
# - label: índice de la label que se quiere buscar.
# - source: lista de la relación de etiquetas.

def __get_start_end(label, source):

    start = 0
    end = 0
    
    try:        
        start = source.index(label)
        end = start + source.count(label)

    except:
        print("label doesn't exist")

    return start, end

In [ ]:
# Devuelve las imagenes y etiquetas de una carpeta especifíca
def __read_images_labels_from_dir(directory, images, labels, shape, as_gray, get_label_from_dir):
    
    #Buscamos fotos en el directorio
    for f in os.listdir(directory):
        # Cargamos archivos con extension .ppm 
        # Obviamos el color y lo cargamos como escala de grises y normalizamos el tamaño
        if f.endswith(".ppm"):
            image = load_image(os.path.join(directory, f), shape, as_gray)
            images.append(image)
            if get_label_from_dir:
                labels.append(int(os.path.basename(directory)))
    
    return images, labels

## Métodos públicos

In [5]:
# Devuelve una imagen
# - path: ruta de la imagen.
# - size: dimensión con la que se cargará la imagen.
# - as_gray: True para cargar la imagen en escala de grises.

def load_image(path, size, as_gray):
    aux = skimage.data.imread(path, as_gray = as_gray)
    return skimage.transform.resize(aux, size, mode='constant')

In [6]:
# Devuelve una lista con los datos del fichero csv.
# - path: ruta hasta el fichero csv.
# - delimeter: Carácter delimitador de campos

def read_csv(path, delimiter):
    file = open(path)
    title_csv = csv.reader(file, delimiter = delimiter)
    return list(title_csv)

In [7]:
# Devuelve una colección con las imágenes y los labels de la ruta
# -datadir: path donde se encuentran la colección de imágenes.
# -shape: Dimensiones con las que se cargarán las imágenes.
# -as_gray: Indica si la imagen se cargará en escala de grises.
# -get_label_from_dir: (defecto True) específica si la categoría se lee del propio directorio
def readDataset(data_dir, shape, as_gray, get_label_from_dir = True):
    images = []
    labels = []
    directories = __get_directories(data_dir)
    
    images, labels = __read_images_labels_from_dir(data_dir, images, labels, shape, as_gray, get_label_from_dir)
    
    for d in directories:
                
        #Buscamos fotos en el directorio
        images_dir = os.path.join(data_dir, d)
        images, labels = __read_images_labels_from_dir(images_dir, images, labels, shape, as_gray, get_label_from_dir)
       
    return images, labels

In [8]:
# Imprime los tamaños de las colecciones
# - images: lista de imágenes precargadas.
# - labels: relación de las imágenes con las categorías a las que pertenecen.
# - np_images: np.array con las imágenes precargadas.
# - np_labels: np.array con las categorías precargadas.
# - environment: string identificativo del entorno.

def print_size_dataset(images, labels, np_images, np_labels, environment):
    print("Total images (" + environment + "): ", len(images))
    print("Total labels (" + environment + "): ", len(set(labels)))
    print("Images shape: ", np_images.shape)
    print("Labels shape: ", np_labels.shape)

In [9]:
# Imprime una matriz 32x32 con los diferentes tipos de señales que
# se van a clasificar. Muestra la primera imagen de cada categoría.
# - images: lista de imágenes precargadas.
# - labels: relación de las imágenes con las categorías a las que pertenecen.
# - titles: lista con los nombres de las categorías.

def print_summary_dataset(images, labels, titles):
    
    #Quitamos los repetidos a los labels
    unique_labels = set(labels)
    
    plt.figure(figsize=(32, 32))
    i = 1
    
    for label in unique_labels:
        
        #Obtenemos la primera imagén de cada label
        image = images[labels.index(label)]
        plt.subplot(8, 8, i)
        plt.axis('off')
        plt.title(titles[i-1][1])
        i += 1
        _ = plt.imshow(image)
        
    plt.show()

In [10]:
# Imprime todas las imágenes de una label especifica
# - label: índice de la label que se quiere buscar.
# - images: lista de imágenes precargadas.
# - source: lista de la relación de etiquetas.
# - titles: lista con las diferentes categorías de labels.

def print_signals(label, images, source, titles):
    
    start, end = __get_start_end(label, source)
    
    if start < end:
        rows = (end - start) / 8

        print("Signal: ", titles[label][1])
        plt.figure(figsize=(10, 10))
        i = 1

        for image in images[start:end]:
            plt.subplot(rows + 1, 8, i)
            plt.axis('off')
            i += 1
            plt.imshow(image)

        plt.show()

In [1]:
# Imprime los atributos de las imágenes de una label especifica.
# - label: índice de la label que se quiere buscar.
# - images: lista de imágenes precargadas.
# - source: lista de la relación de etiquetas.
# - titles: lista con las diferentes categorías de labels.

def print_signals_attributes(label, images, source, titles):
    
    start, end = __get_start_end(label, source)
    
    if start < end:
        print("Signal: ", titles[label][1])
        for image in images[start:end]:
            print("shape: ", image.shape, "\tmin:", image.min(), "\tmax: ", image.max())

In [ ]:
# Devuelve una lista con las categorías de las imágenes de prueba leídas del fichero csv.
# - csv: fichero csv
# - class_column: número de columna que contiene las clases.
# - first_is_header: Indica si la primera fila es una cabecerá.

def get_class_id_array(csv, class_column, first_is_header = True):
    labels = []
    for row in csv:
        if not first_is_header:
            labels.append(int(row[class_column]))
        else:
            first_is_header = False
    
    return labels

In [ ]:
# Devuelve un NP-Array con las images cargadas a un tamaño específico.
def get_array_images_test(images, size):
    images_test = []
    for image_path in images:
        images_test.append(load_image(image_path, size, False))
    return np.array(images_test)